In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
from pandas_datareader import data
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.formula.api as smf


computing the expected return of an asset using Fama-French 3 factor model:<br>

ER=Rf + b1*[ ERm - Rf]+b2*[ SMB ]+b3*[ HML ]<br>

where ER is expected return of asset,<br>

Rf is risk free rate,<br>

ERm is expected return of market,<br>

SMB is small minus big, w hich reflects the excess returns of small versus large companies,<br>

HML is high(book to price) minus low, which reflects the excess returns of value over growth stocks(at least historically).

Macro economic affects the volatility of stock. Some risks are not avoidable and non-diversifable hence, investors should expect higher returns for investing in this asset.

How do we measure sytematic risk?<br>
-We try to see how much this stock is exposed to the market movements.

The Fama-French 3 factor is an extension of the CAPM specifically. In addition to the market factor we have in CAPM, we have two more factors, size and value factors. 

### Stock data

In [2]:
df=pd.read_csv("/Users/USER/Documents/DATASETS/NSE_MONTHLY_Stock_data_2012_2022_WITH_ASI.csv")

In [3]:
df=df[['datetime','ASI','AIRTELAFRI','PRESCO','OKOMUOIL','UCAP','FIDELITYBK','LIVINGTRUST','MEYER']]

In [4]:
df=df.iloc[116:,:]

In [5]:
df['datetime']=pd.to_datetime(df['datetime'])
df=df.set_index('datetime')

In [6]:
df=df.pct_change().dropna(how='all')

In [7]:
#adjusting our data to get monthly returns instead of daily
df=df.resample('1M').last()

In [8]:
df.head()

,ASI,AIRTELAFRI,PRESCO,OKOMUOIL,UCAP,FIDELITYBK,LIVINGTRUST,MEYER
datetime,,,,,,,,
2021-10-31,0.045186,0.090909,0.109034,0.196809,0.027933,0.125506,0.194030,0.000000
2021-11-30,0.028770,0.217949,-0.013483,0.051852,0.048913,-0.089928,0.200000,0.000000
2021-12-31,-0.012292,0.005263,0.000000,0.000000,0.025907,0.007905,0.083333,1.300000
2022-01-31,0.091492,0.330890,0.099658,0.000000,0.111111,0.101961,0.096154,0.195652
2022-02-28,0.016512,-0.008655,0.077162,-0.049296,0.259091,0.067616,0.087719,0.000000


In [9]:
df=df.reset_index()

In [10]:
ff3=pd.read_csv("/Users/USER/Documents/DATASETS/Fama_french_factors.csv")

In [11]:
ff3['datetime']=pd.to_datetime(ff3['datetime'])

In [12]:
ff3

,datetime,HML,SMB,Rf,asi,Mkt-Rf
0,2021-09-30,-0.000229,0.009393,0.000000,0.025537,0.025537
1,2021-10-31,0.039417,0.088441,0.000000,0.045186,0.045186
2,2021-11-30,-0.011932,-0.017114,0.000000,0.028770,0.028770
3,2021-12-31,0.018859,-0.075099,-0.004000,-0.012292,-0.008292
4,2022-01-31,-0.054132,-0.034212,0.000000,0.091492,0.091492
5,2022-02-28,-0.144912,-0.374175,0.000000,0.016512,0.016512
6,2022-03-31,-0.007865,0.061634,-0.297189,-0.009053,0.288136
7,2022-04-30,-0.153147,0.091260,-0.005714,0.056924,0.062638
8,2022-05-31,0.061007,-0.016428,0.419540,0.080546,-0.338995
9,2022-06-30,0.005401,-0.020103,-0.024291,-0.033923,-0.009632


In [13]:
#next, take the average of this values to confirm if they are premiums
market_premium=ff3['Mkt-Rf'].mean()
size_premium=ff3['SMB'].mean()
value_premium=ff3['HML'].mean()
print(market_premium,size_premium,value_premium)

0.020135312823021015 -0.028640474131390047 -0.024753349610243103


In [14]:
data=ff3.merge(df,on='datetime')

In [15]:
data

,datetime,HML,SMB,Rf,asi,Mkt-Rf,ASI,AIRTELAFRI,PRESCO,OKOMUOIL,UCAP,FIDELITYBK,LIVINGTRUST,MEYER
0,2021-10-31,0.039417,0.088441,0.000000,0.045186,0.045186,0.045186,0.090909,0.109034,0.196809,0.027933,0.125506,0.194030,0.000000
1,2021-11-30,-0.011932,-0.017114,0.000000,0.028770,0.028770,0.028770,0.217949,-0.013483,0.051852,0.048913,-0.089928,0.200000,0.000000
2,2021-12-31,0.018859,-0.075099,-0.004000,-0.012292,-0.008292,-0.012292,0.005263,0.000000,0.000000,0.025907,0.007905,0.083333,1.300000
3,2022-01-31,-0.054132,-0.034212,0.000000,0.091492,0.091492,0.091492,0.330890,0.099658,0.000000,0.111111,0.101961,0.096154,0.195652
4,2022-02-28,-0.144912,-0.374175,0.000000,0.016512,0.016512,0.016512,-0.008655,0.077162,-0.049296,0.259091,0.067616,0.087719,0.000000
5,2022-03-31,-0.007865,0.061634,-0.297189,-0.009053,0.288136,-0.009053,0.000317,0.278846,0.108889,-0.093863,0.066667,0.000000,0.200000
6,2022-04-30,-0.153147,0.091260,-0.005714,0.056924,0.062638,0.056924,0.099651,0.078195,-0.018036,0.071713,0.187500,0.000000,3.545455
7,2022-05-31,0.061007,-0.016428,0.419540,0.080546,-0.338995,0.080546,0.274892,0.129707,0.462585,-0.033457,-0.105263,-0.080645,0.033333
8,2022-06-30,0.005401,-0.020103,-0.024291,-0.033923,-0.009632,-0.033923,-0.019694,0.018519,0.008837,-0.019231,0.005882,0.052632,-0.187097


In [16]:
#get the excess return for Amazon
data['ASI_Excess']=data['ASI']-data['Rf']
data['FIDELITYBK_Excess']=data['FIDELITYBK']-data['Rf']
data['PRESCO_Excess']=data['PRESCO']-data['Rf']
data['OKOMUOIL_Excess']=data['OKOMUOIL']-data['Rf']
data['UCAP_Excess']=data['UCAP']-data['Rf']
data['LIVINGTRUST_Excess']=data['LIVINGTRUST']-data['Rf']
data['MEYER_Excess']=data['MEYER']-data['Rf']
data['AIRTELAFRI_Excess']=data['AIRTELAFRI']-data['Rf']

In [17]:
data

,datetime,HML,SMB,Rf,asi,Mkt-Rf,ASI,AIRTELAFRI,PRESCO,OKOMUOIL,...,LIVINGTRUST,MEYER,ASI_Excess,FIDELITYBK_Excess,PRESCO_Excess,OKOMUOIL_Excess,UCAP_Excess,LIVINGTRUST_Excess,MEYER_Excess,AIRTELAFRI_Excess
0,2021-10-31,0.039417,0.088441,0.000000,0.045186,0.045186,0.045186,0.090909,0.109034,0.196809,...,0.194030,0.000000,0.045186,0.125506,0.109034,0.196809,0.027933,0.194030,0.000000,0.090909
1,2021-11-30,-0.011932,-0.017114,0.000000,0.028770,0.028770,0.028770,0.217949,-0.013483,0.051852,...,0.200000,0.000000,0.028770,-0.089928,-0.013483,0.051852,0.048913,0.200000,0.000000,0.217949
2,2021-12-31,0.018859,-0.075099,-0.004000,-0.012292,-0.008292,-0.012292,0.005263,0.000000,0.000000,...,0.083333,1.300000,-0.008292,0.011905,0.004000,0.004000,0.029907,0.087333,1.304000,0.009263
3,2022-01-31,-0.054132,-0.034212,0.000000,0.091492,0.091492,0.091492,0.330890,0.099658,0.000000,...,0.096154,0.195652,0.091492,0.101961,0.099658,0.000000,0.111111,0.096154,0.195652,0.330890
4,2022-02-28,-0.144912,-0.374175,0.000000,0.016512,0.016512,0.016512,-0.008655,0.077162,-0.049296,...,0.087719,0.000000,0.016512,0.067616,0.077162,-0.049296,0.259091,0.087719,0.000000,-0.008655
5,2022-03-31,-0.007865,0.061634,-0.297189,-0.009053,0.288136,-0.009053,0.000317,0.278846,0.108889,...,0.000000,0.200000,0.288136,0.363855,0.576035,0.406078,0.203326,0.297189,0.497189,0.297506
6,2022-04-30,-0.153147,0.091260,-0.005714,0.056924,0.062638,0.056924,0.099651,0.078195,-0.018036,...,0.000000,3.545455,0.062638,0.193214,0.083910,-0.012322,0.077427,0.005714,3.551169,0.105365
7,2022-05-31,0.061007,-0.016428,0.419540,0.080546,-0.338995,0.080546,0.274892,0.129707,0.462585,...,-0.080645,0.033333,-0.338995,-0.524803,-0.289833,0.043045,-0.452997,-0.500185,-0.386207,-0.144648
8,2022-06-30,0.005401,-0.020103,-0.024291,-0.033923,-0.009632,-0.033923,-0.019694,0.018519,0.008837,...,0.052632,-0.187097,-0.009632,0.030174,0.042810,0.033129,0.005061,0.076923,-0.162805,0.004597


In [18]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['MEYER_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_meyer=model.fit()
results_meyer.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           MEYER_Excess   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     3.593
Date:                Sun, 02 Oct 2022   Prob (F-statistic):              0.101
Time:                        14:14:40   Log-Likelihood:                -8.8922
No. Observations:                   9   AIC:                             25.78
Df Residuals:                       5   BIC:                             26.57
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3815      0.313      1.220      0.277      -0.422       1.185
Mkt-Rf        -1.2690      2.163     -0.587      0.583      -6.829       4.291
SMB            6.1119      2.562      2.385      0.063      -0.474      12.698
HML          -14.5422      4.962     -2.931      0.033     -27.296      -1.788
==============================================================================
Omnibus:                       12.722   Durbin-Watson:                   2.752
Prob(Omnibus):                  0.002   Jarque-Bera (JB):                5.313
Skew:                           1.670   Prob(JB):                       0.0702
Kurtosis:                       4.736   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
#saving parameters
intercept_meyer, beta_market_meyer, beta_size_meyer, beta_value_meyer=results_meyer.params
intercept_meyer, beta_market_meyer, beta_size_meyer, beta_value_meyer


(0.3815134720702415,
 -1.2689921445629577,
 6.111885250496938,
 -14.542228350868022)

In [20]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['PRESCO_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_presco=model.fit()
results_presco.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          PRESCO_Excess   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     13.05
Date:                Sun, 02 Oct 2022   Prob (F-statistic):            0.00844
Time:                        14:14:40   Log-Likelihood:                 11.042
No. Observations:                   9   AIC:                            -14.08
Df Residuals:                       5   BIC:                            -13.30
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0600      0.034      1.758      0.139      -0.028       0.148
Mkt-Rf         1.3653      0.236      5.783      0.002       0.758       1.972
SMB           -0.0555      0.280     -0.198      0.851      -0.774       0.663
HML            0.4342      0.542      0.802      0.459      -0.958       1.826
==============================================================================
Omnibus:                        0.368   Durbin-Watson:                   1.259
Prob(Omnibus):                  0.832   Jarque-Bera (JB):                0.416
Skew:                           0.345   Prob(JB):                        0.812
Kurtosis:                       2.204   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
#saving parameters
intercept_presco, beta_market_presco, beta_size_presco, beta_value_presco=results_presco.params
intercept_presco, beta_market_presco, beta_size_presco, beta_value_presco


(0.060024549003857265,
 1.365263697608041,
 -0.055490188175375425,
 0.43418727070568935)

In [22]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['OKOMUOIL_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_okomuoil=model.fit()
results_okomuoil.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        OKOMUOIL_Excess   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     3.479
Date:                Sun, 02 Oct 2022   Prob (F-statistic):              0.107
Time:                        14:14:40   Log-Likelihood:                 10.394
No. Observations:                   9   AIC:                            -12.79
Df Residuals:                       5   BIC:                            -12.00
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0973      0.037      2.651      0.045       0.003       0.192
Mkt-Rf         0.6174      0.254      2.434      0.059      -0.035       1.270
SMB            0.1490      0.301      0.496      0.641      -0.624       0.922
HML            1.0780      0.582      1.852      0.123      -0.418       2.574
==============================================================================
Omnibus:                        1.021   Durbin-Watson:                   1.542
Prob(Omnibus):                  0.600   Jarque-Bera (JB):                0.680
Skew:                           0.291   Prob(JB):                        0.712
Kurtosis:                       1.785   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
#saving parameters
intercept_okomuoil, beta_market_okomuoil, beta_size_okomuoil, beta_value_okomuoil=results_okomuoil.params
intercept_okomuoil, beta_market_okomuoil, beta_size_okomuoil, beta_value_okomuoil


(0.09726955494909882,
 0.6174401627862127,
 0.1490320867891616,
 1.0780398607303752)

In [24]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['UCAP_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_ucap=model.fit()
results_ucap.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            UCAP_Excess   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     43.91
Date:                Sun, 02 Oct 2022   Prob (F-statistic):           0.000514
Time:                        14:14:41   Log-Likelihood:                 17.057
No. Observations:                   9   AIC:                            -26.11
Df Residuals:                       5   BIC:                            -25.33
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0158      0.017     -0.904      0.407      -0.061       0.029
Mkt-Rf         1.0864      0.121      8.978      0.000       0.775       1.397
SMB           -0.5152      0.143     -3.594      0.016      -0.884      -0.147
HML           -0.4399      0.278     -1.584      0.174      -1.153       0.274
==============================================================================
Omnibus:                        0.576   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.750   Jarque-Bera (JB):                0.447
Skew:                          -0.435   Prob(JB):                        0.800
Kurtosis:                       2.341   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
#saving parameters
intercept_ucap, beta_market_ucap, beta_size_ucap, beta_value_ucap=results_ucap.params
intercept_ucap, beta_market_ucap, beta_size_ucap, beta_value_ucap

(-0.015824222849159242,
 1.0864085930831464,
 -0.5152326500708506,
 -0.4398502882010295)

In [26]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['LIVINGTRUST_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_livingtrust=model.fit()
results_livingtrust.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     LIVINGTRUST_Excess   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     10.90
Date:                Sun, 02 Oct 2022   Prob (F-statistic):             0.0124
Time:                        14:14:41   Log-Likelihood:                 10.179
No. Observations:                   9   AIC:                            -12.36
Df Residuals:                       5   BIC:                            -11.57
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0399      0.038      1.062      0.337      -0.057       0.137
Mkt-Rf         1.4255      0.260      5.485      0.003       0.757       2.093
SMB           -0.3187      0.308     -1.035      0.348      -1.110       0.473
HML            0.6439      0.596      1.080      0.329      -0.888       2.176
==============================================================================
Omnibus:                        0.229   Durbin-Watson:                   1.520
Prob(Omnibus):                  0.892   Jarque-Bera (JB):                0.386
Skew:                          -0.151   Prob(JB):                        0.824
Kurtosis:                       2.031   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
#saving parameters
intercept_livingtrust, beta_market_livingtrust, beta_size_livingtrust, beta_value_livingtrust=results_livingtrust.params
intercept_livingtrust, beta_market_livingtrust, beta_size_livingtrust, beta_value_livingtrust


(0.03991463422583062,
 1.4254597184374984,
 -0.3186727962801727,
 0.6438836478380728)

In [28]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['AIRTELAFRI_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_airtelafri=model.fit()
results_airtelafri.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      AIRTELAFRI_Excess   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     3.720
Date:                Sun, 02 Oct 2022   Prob (F-statistic):             0.0957
Time:                        14:14:41   Log-Likelihood:                 9.7446
No. Observations:                   9   AIC:                            -11.49
Df Residuals:                       5   BIC:                            -10.70
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0927      0.039      2.349      0.066      -0.009       0.194
Mkt-Rf         0.7527      0.273      2.760      0.040       0.052       1.454
SMB            0.2073      0.323      0.642      0.549      -0.623       1.038
HML            0.0069      0.626      0.011      0.992      -1.601       1.615
==============================================================================
Omnibus:                        4.449   Durbin-Watson:                   2.850
Prob(Omnibus):                  0.108   Jarque-Bera (JB):                2.008
Skew:                           1.156   Prob(JB):                        0.366
Kurtosis:                       2.901   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
#saving parameters
intercept_airtelafri, beta_market_airtelafri, beta_size_airtelafri, beta_value_airtelafri=results_airtelafri.params
intercept_airtelafri, beta_market_airtelafri, beta_size_airtelafri, beta_value_airtelafri


(0.09265088548815559,
 0.7527233822969195,
 0.20730603703505673,
 0.006880648395175695)

In [30]:
X=data[['Mkt-Rf','SMB','HML']]
y=data['FIDELITYBK_Excess']

X1=sm.add_constant(X)

model=sm.OLS(y,X1)

results_fidelitybk=model.fit()
results_fidelitybk.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      FIDELITYBK_Excess   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     23.09
Date:                Sun, 02 Oct 2022   Prob (F-statistic):            0.00234
Time:                        14:14:41   Log-Likelihood:                 12.596
No. Observations:                   9   AIC:                            -17.19
Df Residuals:                       5   BIC:                            -16.40
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0057      0.029     -0.199      0.850      -0.080       0.068
Mkt-Rf         1.3383      0.199      6.737      0.001       0.828       1.849
SMB            0.1175      0.235      0.499      0.639      -0.487       0.722
HML           -0.5274      0.456     -1.157      0.299      -1.699       0.644
==============================================================================
Omnibus:                        1.967   Durbin-Watson:                   3.109
Prob(Omnibus):                  0.374   Jarque-Bera (JB):                0.750
Skew:                          -0.701   Prob(JB):                        0.687
Kurtosis:                       2.813   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
#saving parameters
intercept_fidelitybk, beta_market_fidelitybk, beta_size_fidelitybk, beta_value_fidelitybk=results_fidelitybk.params
intercept_fidelitybk, beta_market_fidelitybk, beta_size_fidelitybk, beta_value_fidelitybk

(-0.005720210600009859,
 1.3383039642513008,
 0.11748178311269787,
 -0.5274357134394693)

In [32]:
risk_free_rate=data['Rf'].mean()

In [33]:
#EXPECTED RETURNS

meyer_return=risk_free_rate + (beta_market_meyer*market_premium) + (beta_size_meyer*size_premium) + (beta_value_meyer*value_premium)

presco_return=risk_free_rate + (beta_market_presco*market_premium) + (beta_size_presco*size_premium) + (beta_value_presco*value_premium)

okomuoil_return=risk_free_rate + (beta_market_okomuoil*market_premium) + (beta_size_okomuoil*size_premium) + (beta_value_okomuoil*value_premium)

ucap_return=risk_free_rate + (beta_market_ucap*market_premium) + (beta_size_ucap*size_premium) + (beta_value_ucap*value_premium)

airtelafri_return=risk_free_rate + (beta_market_airtelafri*market_premium) + (beta_size_airtelafri*size_premium) + (beta_value_airtelafri*value_premium)

livingtrust_return=risk_free_rate + (beta_market_livingtrust*market_premium) + (beta_size_livingtrust*size_premium) + (beta_value_livingtrust*value_premium)

fidelitybk_return=risk_free_rate + (beta_market_fidelitybk*market_premium) + (beta_size_fidelitybk*size_premium) + (beta_value_fidelitybk*value_premium)




In [34]:

print('The monthly expected return of MEYER stock is',meyer_return*100,'%')
print('The yearly expected return of MEYER stock is',(meyer_return*12)*100,'%')
print('_____________________________________________________________________________________')


print('The monthly expected return of PRESCO stock is',presco_return*100,'%')
print('The yearly expected return of PRESCO stock is',(presco_return*12)*100,'%')
print('_____________________________________________________________________________________')


print('The monthly expected return of OKOMUOIL stock is',okomuoil_return*100,'%')
print('The yearly expected return of OKOMUOIL stock is',(okomuoil_return*12)*100,'%')
print('_____________________________________________________________________________________')


print('The monthly expected return of UCAP stock is',ucap_return*100,'%')
print('The yearly expected return of UCAP stock is',(ucap_return*12)*100,'%')
print('_____________________________________________________________________________________')


print('The monthly expected return of FIDELITYBK stock is',fidelitybk_return*100,'%')
print('The yearly expected return of FIDELITYBK stock is',(fidelitybk_return*12)*100,'%')
print('_____________________________________________________________________________________')


print('The monthly expected return of AIRTELAFRI stock is',airtelafri_return*100,'%')
print('The yearly expected return of AIRTELAFRI stock is',(airtelafri_return*12)*100,'%')
print('_____________________________________________________________________________________')

print('The monthly expected return of LIVINGTRUST stock is',livingtrust_return*100,'%')
print('The yearly expected return of LIVINGTRUST stock is',(livingtrust_return*12)*100,'%')
print('_____________________________________________________________________________________')

The monthly expected return of MEYER stock is 16.918620517774325 %
The yearly expected return of MEYER stock is 203.02344621329192 %
_____________________________________________________________________________________
The monthly expected return of PRESCO stock is 2.8147875536471383 %
The yearly expected return of PRESCO stock is 33.77745064376566 %
_____________________________________________________________________________________
The monthly expected return of OKOMUOIL stock is -0.8704908457339909 %
The yearly expected return of OKOMUOIL stock is -10.445890148807893 %
_____________________________________________________________________________________
The monthly expected return of UCAP stock is 5.733564012968681 %
The yearly expected return of UCAP stock is 68.80276815562416 %
_____________________________________________________________________________________
The monthly expected return of FIDELITYBK stock is 4.645442352236696 %
The yearly expected return of FIDELITYBK stock i